<span style = "font-size:250%">
<center> Implementation of HHL Algorithm </center>
</span>

<center>
    <img src="https://qiskit.org/textbook/ch-applications/images/hhlcircuit.png" width = "50%" height = "50%">
    </center>


<span style = "font-size:150%">
1. QPE 파트에 대한 구현 
</span>

<center>
    <img src="https://qiskit.org/textbook/ch-algorithms/images/qpe_tex_qz.png" width = "50%" height = "50%">
    </center>

In [137]:
#QPE에 필요한 패키지 불러오기
#import qiskit
#scipy.linalg : 선형대수적 계산을 위한 패키지 
#https://docs.scipy.org/doc/scipy/reference/linalg.html
#from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
#from qiskit.quantum_info.operators import Operator
#from qiskit.extensions import UnitaryGate
#from qiskit.circuit.add_control import add_control

#from qiskit.visualization import plot_histogram
#import numpy as np
#from qiskit import QuantumCircuit, QuantumRegister
#from qiskit.circuit.library.arithmetic.piecewise_chebyshev import PiecewiseChebyshev
#from qiskit import IBMQ, Aer, transpile, assemble

In [142]:
#run.py

import numpy as np
from qiskit.extensions import UnitaryGate
from qiskit.circuit.add_control import add_control
from scipy.linalg import expm

#참고 논문 :Low Complexity Quantum Matrix Inversion A기gorithm for non-Hermitian Matrices

A = np.array([[2,-1],[1,4]]) #non-Hermitian인 경우의 행렬에 대한 저장
b = np.array([1,1]) 

if np.allclose(A,A.T) == False:
    print("Given A matrice is not Hermitian.")
    print("Given Matrices will be transformed into Hermitian formation.")
    A = np.vstack((np.hstack((np.zeros_like(A),A)),np.hstack((A.T, np.zeros_like(A))))) # Hermitian의 꼴로 바꿈
    b = np.hstack((b,np.zeros_like((np.shape(A)[0]-np.shape(b)[0],1))))

#A의 shape와 동일한 zero array를 생성하고, A의 왼쪽에 배치, horizontal 방향도 마찬가지.

i = complex(0,1) #complex(real part, imaginary part)
t = np.pi*2/16

U = expm(i*A*t) #여기서 A가 행렬로 주어졌기 때문에, 행렬을 exp에 올리기 위해서는 expm이라는 scipy 패키지가 필요함.
U_gate = UnitaryGate(U) #위에서 구성한 U라는 행렬로써 Unitary gate를 구성할 수 있음. (4*4) 행렬
CU = add_control(U_gate,1,ctrl_state=None, label="CU") 
#CU라는 게이트 이름을 label에 저장
#control 되는 경우의 state를 지정 -> 해당사항 없음
#두번째 인자는 컨트롤 큐빗의 개수를 지정함.

n_l = 3 #QPE 상에서 n_ㅣ는 하다마드로 초기화 되는 부분 
n_b = int(np.log2(U.shape[0])) 
#Ax =b의 꼴이고, b는 4*1의 shape이므로, A의 행의 개수와 동일함. 따라서, U의 행렬의 행의 개수와 동일함.
#행의 개수에 log2를 취하면 필요한 n_b의 값을 구할 수 있음.

delta = 1/16*(2**(n_l-1))
backend = Aer.get_backend('aer_simulator')
shots = 8192

Given A matrice is not Hermitian.
Given Matrices will be transformed into Hermitian formation.


In [157]:
#circuit_parts.py

from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal
from qiskit.circuit.library.arithmetic.piecewise_chebyshev import PiecewiseChebyshev
from qiskit import QuantumCircuit, QuantumRegister,Aer
from qiskit import IBMQ, Aer, transpile, assemble
from qiskit.visualization import plot_histogram

import numpy as np

def qft_dagger(n_l):

# <qft를 구현하는 과정에 있어서 SWAP gate에 대한 참고사항>

# SWAP 게이트를 걸어주는 목적은 qiskit은 qubit을 반대방향으로 읽기 때문임.
# 하지만, SWAP 게이트를 위와 같은 이유로 걸어주게 된다고 하면, 
# HHL 알고리즘 상에서 Eigeninversion 단계에서 문제가 생기게 됨. 
# 즉, Eigeninversion에서는 SWAP이 된 상태를 인지하지 못하고 연산을 실시하여 잘못된 연산이 나오게 됨.

    """n-qubit QFTdagger the first n qubits in circ"""
    nl_rg = QuantumRegister(n_l, "l")
    qc = QuantumCircuit(nl_rg)
    # Don't forget the Swaps!
    #QFT의 역연산은 곧 QFT_dagger임을 기억하자.
        
    for j in reversed(range(n_l)):
        qc.h(j)
        for m in reversed(range(j)):
                qc.cp(-np.pi/float(2**(j-m)), m, j)
    qc.name = "QFT†"
    #display(qc.draw(output = 'mpl'))
    return qc
    
def QPE(n_l,n_b,CU):
    #circuit initialization for HHL
    nl_rg = QuantumRegister(n_l, "l")
    nb_rg = QuantumRegister(n_b, "b")
    #QuantumRegister(size=None, name=None, bits=None) 
    qc = QuantumCircuit(nl_rg,nb_rg)
    #display(qc.draw(output = 'mpl'))
    qc.h(nl_rg)
    qc.barrier(nl_rg[:]+nb_rg[:])
    for l in range(n_l):
        for power in range(2**(l)):
            qc.append(CU, [nl_rg[l],nb_rg[0],nb_rg[1]]) 
            #첫번째 큐비트는 2^0번, 이후 2^n꼴로 돌아가게 설계됨.
            #https://qiskit.org/documentation/stubs/qiskit.circuit.ControlledGate.html append의 예제.
            #즉, append의 첫번째 인자는 gate, 두번쨰 인자의 첫번째 요소는 control qubit, 이후 인자의 요소는 target qubit.

    qc.barrier(nl_rg[:]+nb_rg[:])
    qc.append(qft_dagger(n_l),nl_rg[:])
    qc.barrier(nl_rg[:]+nb_rg[:])
    qc.name = "QPE"
    #display(qc.draw(output = 'mpl'))
    return qc
    
def QPE_dagger(n_l,n_b,CU):
    qc = QPE(n_l,n_b,CU)
    qc = qc.inverse()
    #여기서 inverse함수는 모든 rotation 각도까지도 반대로 입력해줌을 확인하였음.
    #QPE dagger는 그저, QPE의 역과정이라고 생각하면 된다. 단, 각도는 반대방향이어야 함.
    #따라서 여기서 inverse함수를 이용하여 QPE의 역과정, 즉, QPE dagger를 실시하였음
    qc.name = 'QPE†'
    return qc

def Eigenvalue_inversion(n_l,chevyshev = False):

    #Chevyshev 근사를 이용한 풀이방법.
    #Qiskit에서 제공한 HHL 알고리즘 상에서는 Chevyshev 근사를 이용한 부분이 있었다.
    #일단 Chevyshev 근사를 이용하는 경우, 기존 Taylor 근사보다 훨씬 빠르게 급수에 수렴한다는 장점이 있다.
    #참고 문헌 : https://freshrimpsushi.github.io/posts/chebyshev-expansion/
    #여기서는 위의 표현한 cos(theta)에 대한 표현을 Chevyshev근사를 이용해 theta값을 알아내겠다는 접근방법이다.
    #하지만, 근사결과가 좋지 못하다는 점 때문에 Chevyshev 근사를 이용하는 대신에 직접 exact한 theta값을 알아내는 ExactReciprocal을 이용하였다.

    if chevyshev == True:
        print("Maybe using Chevyshev approximation is not accurate.")
        #Using Chebychev Approx. (not recommended!)
        nl_rg = QuantumRegister(n_l, "l")
        na_rg = QuantumRegister(n_l, "a")
        nf_rg = QuantumRegister(1, "f")
        qc = QuantumCircuit(nl_rg, na_rg, nf_rg)

        f_x, degree, breakpoints, num_state_qubits = lambda x: np.arcsin(1 / x), 2, [1,2,3,4], n_l
        #degree : 함수를 polynomial로 근사할 떄, 최고차항 정의
        #breakpoints는 구간을 나누는 느낌. : 근사를 할 떄, 다항식을 어떤 구간에서 나눠서 사용할 지
        #l : eigenvalue를 표현
        #f : rotation
        #a : ancila
        pw_approximation = PiecewiseChebyshev(f_x, degree, breakpoints, num_state_qubits)
        pw_approximation._build()
        qc.append(pw_approximation,nl_rg[:]+[nf_rg[0]]+na_rg[:]) #range(nl*2+1))
        qc.name = 'Chevyshev_inversion'
        return qc

    else:
        qc = ExactReciprocal(n_l, delta, neg_vals = True)
        qc.name = 'Reciprocal_inversion'
        return qc

In [ ]:
#measurement.py

def measurement(qc,n_l,n_b,CU,backend):
    
    t = transpile(qc, backend)
    qobj = assemble(t, shots=shots)
    results = backend.run(qobj).result()
    answer = results.get_counts()    
    plot_histogram(answer, title="Output Histogram").savefig('output_histogram.png',facecolor='#eeeeee')

    return answer

<Eigenvalue inversion에 대한 이해>

Ry gate의 행렬을 |0> 인 state에 적용시켜보도록하자.

이 경우, 손으로 직접 풀어보게 된다면 cos(theta) |0> + sin(theta) |1>을 얻을 수가 있다.

Eigen inversion 파트에서는 Ry 게이트를 이용하여, 위 식의 cos(theta) = sqrt(1-c^2/lambda_j^2)와 같은 꼴로 두겠다는 의미이다.


In [146]:
#normalization.py

import numpy as np

#양자 회로를 통해서 얻어진 결과(dictionary)를 통해서 normalize된 결과 벡터 x를 구하는 함수
def normalize_vector(answer, nb):
    #nb register에서 얻어질 수 있는 상태들을 dictionary의 key의 형태로 만들어 저장한다.
    possible_states = []
    for s in range(2**(nb)):
        possible_states.append(format(s, "b").zfill(nb))
    #print(answer)
    #flag register를 측정한 결과가 1이 나온 경우에 대해서 nb register의 결과를 순서대로 추가한다.
    available_result = []
    for i in possible_states:
        for key in answer.keys():
        
            if key[0:2] == i:
                if int(key[-1]) == 1:
                    available_result.append(answer[key])
                else:
                    pass
            else:
                pass
    #확률 분포를 상태 벡터의 형식으로 바꾸기 위해서 제곱근을 취한다.
    available_result = np.sqrt(np.array(available_result))
    #벡터의 크기가 1이 되도록 normalize해준다.
    normalized_result = available_result/np.linalg.norm(available_result)
    return normalized_result

In [158]:
#hhl/my_hhl.py

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit import QuantumCircuit, QuantumRegister,Aer


#위에서 정의한 모든 함수들을 한데 불러와서 회로를 구성하고 이를 지정한 backend로써 실행시키는 함수.
def My_HHL(CU,b,n_l,n_b,backend,details = True,chevyshev = False):
    #b_sol = np.hstack((b,np.zeros_like((np.shape(A)[0]-np.shape(b)[0],1))))

    #circuit initialization
    n_f = 1
    nb = int(np.log2(b.shape))
    nl_rg = QuantumRegister(n_l, "l")
    nb_rg = QuantumRegister(n_b, "b")
    na_rg = QuantumRegister(n_l, "a")
    nf_rg = QuantumRegister(n_f, "f")
    
    cf = ClassicalRegister(n_f, "classical_f")
    cb = ClassicalRegister(n_b, "classical_b")

    qc = QuantumCircuit(nf_rg,nl_rg, nb_rg, na_rg, cf, cb)
    #qc.isometry(b/np.linalg.norm(b), list(range(nb)), None)
    qc.h(nb_rg[0])
    qc.barrier(nf_rg,nl_rg,nb_rg)

    
    if details == True:
        qc = qc.compose(QPE(n_l,n_b,CU),nl_rg[:]+nb_rg[:]) 
        qc = qc.compose(Eigenvalue_inversion(n_l,chevyshev),[nl_rg[2]]+[nl_rg[1]]+[nl_rg[0]]+nf_rg[:])
        qc = qc.compose(QPE_dagger(n_l,n_b,CU),nl_rg[:]+nb_rg[:])
        qc.barrier(nf_rg[:]+nl_rg[:]+nb_rg[:])
        qc.measure(nf_rg,cf)
        qc.measure(nb_rg,cb)
        answer = measurement(qc,n_l,n_b,CU,backend)
        qc.draw(output = 'mpl').savefig('qc_HHL')

    else:
        qc.append(QPE(n_l,n_b,CU),nl_rg[:]+nb_rg[:])
        qc.append(Eigenvalue_inversion(n_l),[nl_rg[2]]+[nl_rg[1]]+[nl_rg[0]]+nf_rg[:])
        qc.append(QPE_dagger(n_l,n_b,CU),nl_rg[:]+nb_rg[:])
        qc.barrier(nf_rg[:]+nl_rg[:]+nb_rg[:])
        qc.measure(nf_rg,cf)
        qc.measure(nb_rg,cb)
        answer = measurement(qc,n_l,n_b,CU,backend)
        qc.draw(output = 'mpl').savefig('qc_HHL')

    #Obtaining Normalized answer
    normalized_result = normalize_vector(answer, n_b)
    
    #Obtaining Real Answer
    constant = b/(A @ normalized_result)
    constant = (constant[0]+constant[1])/2
    constant = np.mean(constant)

    print('<My_HHL>')
    print('Normalized Answer : {0}'.format(normalized_result)) 
    print('Un-normalized Answer : {0}'.format(normalized_result * constant))
    print('Normalize Constant: ' ,constant)

    return [normalized_result,normalized_result * constant]


In [ ]:
#circuit/qiskit_hhl.py

def qiskit_HHL(A,b):

    backend = Aer.get_backend('aer_simulator')
    #qiskit HHL 코드를 불러옴
    hhl = HHL(quantum_instance=backend)
    #A, b에 대해서 HHL 회로를 구성
    solution = hhl.solve(A, b)
    #만들어진 회로를 그림으로 저장
    solution.state.draw("mpl").savefig("./outputs/HHL_circuit_qiskit.png")
    #연산된 상태를 상태 벡터의 형태로 결과를 얻음
    naive_sv = Statevector(solution.state).data
    #qubit수를 확인
    num_qubit = solution.state.num_qubits
    #상태 벡터에서 필요한 상태만을 골라서 저장함
    naive_full_vector = np.array([naive_sv[2**(num_qubit-1)+i] for i in range(len(b_sol))])
    #실수 부분만 취함
    naive_full_vector = np.real(naive_full_vector)
    #얻어진 벡터를 normalize하여 반환
    normalized_result = naive_full_vector/np.linalg.norm(naive_full_vector)

    constant = b/(A @ normalized_result)
    constant = (constant[0]+constant[1])/2
    constant = np.mean(constant)

    print('<Qiskit_HHL>')
    print('Normalized Qiskit Answer : {0}'.format(normalized_result))
    print('Un-normalized Qiskit Answer : {0}'.format(normalized_result * constant))
    print('Normalize Constant: ' ,constant)

    return [normalized_result,normalized_result * constant]

In [159]:
#circuit/classical_hhl.py

def classical_HHL(A,b):
    
    sol = NumPyLinearSolver().solve(A, b)
    sol_state = sol.state
    norm_state = sol_state/np.linalg.norm(sol_state)

    print('<Classical case using Numpy>')

    if np.shape(b)[0] == 2:
        sol_state = np.pad(sol_state,(2,0))
        norm_state = np.pad(norm_state,(2,0))

    print('Un-normalized Classical Numpy answer : {0}'.format(sol_state,(2,0)))
    print('Normalized Classical Numpy answer : {0}'.format(norm_state,(2,0)))
    
    return [norm_state,sol_state]

In [153]:
#output_show.py

My_HHL_result = My_HHL(CU,b,n_l,n_b,backend,details = True,chevyshev = False)
print("\n")
qiskit_result = qiskit_HHL(A,b)
print("\n")
classical_result = classical_HHL(A,b)
print("\n")

#For normalized answer
print("<Un - normalized Case Comparision>")
print('Qiskit Error : {0}'.format(np.linalg.norm(qiskit_result[1]-classical_result[1])))
print('My HHL Error : {0}'.format(np.linalg.norm(My_HHL_result[1]-classical_result[1])))
print("\n")

print("<Normalized Case Comparision>")
print('Qiskit Error : {0}'.format(np.linalg.norm(qiskit_result[0]-classical_result[0])))
print('My HHL Error : {0}'.format(np.linalg.norm(My_HHL_result[0]-classical_result[0])))

<My_HHL>
Normalized Answer : [0.10225669 0.07056382 0.97436009 0.18758124]
Un-normalized Answer : [0.05867644 0.04049059 0.55910262 0.10763696]
Normalize Constant:  0.5738151867943297


<Qiskit_HHL>
Normalized Qiskit Answer : [ 1.06475857e-16 -4.16460961e-16  9.84477735e-01  1.75509512e-01]
Un-normalized Qiskit Answer : [ 6.12515258e-17 -2.39574211e-16  5.66332738e-01  1.00963972e-01]
Normalize Constant:  0.5752621092062347


<Classical case using Numpy>
Un-normalized Classical Numpy answer : [0.         0.         0.55555556 0.11111111]
Normalized Classical Numpy answer : [0.         0.         0.98058068 0.19611614]


<Un - normalized Case Comparision>
Qiskit Error : 0.014802435887870787
My HHL Error : 0.07146372923545535


<Normalized Case Comparision>
Qiskit Error : 0.02097188616902724
My HHL Error : 0.12468851068707074
